John-Henry Pezzuto - HW8

### Load Packages & Data

In [3]:
import pandas as pd
import numpy as np
drink = pd.read_csv('data/strongdrink.txt')

#### (a) Create a scatterplot of the data

#### (b) Fit a multinomial logistic model

#### (c) Use sklearn.ensemble.RandomForestClassifier

#### (d) Use sklearn.svm.SVC to fit a support vector machines model

#### (e) Use sklearn.neural network.MLPClassifier

#### (f) Which is the best predictor